# Preprocessing the raw data and populating MySQL databse

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json

In [2]:
df = pd.read_csv("customer_support_tickets.csv")

In [3]:
df.shape

(8469, 17)

## Raw data has unique user for every ticket. So sampling 500 users out of all users

In [4]:
users = df[["Customer Name", "Customer Email", "Customer Age", "Customer Gender"]].sample(500)

In [5]:
users.sample()

,Customer Name,Customer Email,Customer Age,Customer Gender
7134,Jerry Rodgers,eric03@example.org,56,Male


## Reassign sampled users to tickets

In [6]:
df[["Customer Name", "Customer Email", "Customer Age", "Customer Gender"]] = users.sample(len(df), replace=True, ignore_index=True)

In [7]:
df["Date of Purchase"] = pd.to_datetime(df["Date of Purchase"])

In [8]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Scott Farley,gomezbeth@example.net,67,Female,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,James Rhodes,brent43@example.net,38,Other,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Debbie Buchanan,martinnicholas@example.net,66,Male,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Justin Buchanan,riverajonathan@example.org,51,Other,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Anita Green,kathleenduke@example.com,41,Male,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Ticket ID                     8469 non-null   int64         
 1   Customer Name                 8469 non-null   object        
 2   Customer Email                8469 non-null   object        
 3   Customer Age                  8469 non-null   int64         
 4   Customer Gender               8469 non-null   object        
 5   Product Purchased             8469 non-null   object        
 6   Date of Purchase              8469 non-null   datetime64[ns]
 7   Ticket Type                   8469 non-null   object        
 8   Ticket Subject                8469 non-null   object        
 9   Ticket Description            8469 non-null   object        
 10  Ticket Status                 8469 non-null   object        
 11  Resolution                    

## Load Databse Credentials

In [10]:
with open('./credentials.json') as fh:
    credentials = json.load(fh)

In [11]:
user = credentials["user"]
password = credentials["password"]
hostname = credentials["hostname"]
database = credentials["database"]

## Create connection

In [12]:
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{hostname}/ctms")

## Insert data into the table

In [13]:
df.to_sql(name='customer_support_tickets', con=engine, if_exists="replace", index=False)

8469

In [14]:
engine.dispose()